In [1]:
import pandas as pd
import os
import sys
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto(device_count = {'CPU': 1}, intra_op_parallelism_threads=7, inter_op_parallelism_threads=1)
session = tf.Session(config=config)
K.set_session(session)
import keras

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import PconvNetPolimi as pcnp
import numpy as np
import gene4cancer as gc
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

%matplotlib inline

matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
matplotlib.rcParams['font.size'] = 22
matplotlib.rcParams['xtick.labelsize'] = 22
matplotlib.rcParams['ytick.labelsize'] = 22
matplotlib.rcParams['axes.labelsize'] = 30
matplotlib.rcParams['axes.titlesize'] = 30

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [25]:
from keras.models import Model
from keras.layers import (Lambda, Flatten, Dropout, Dense, Input, BatchNormalization)
from keras.backend import floatx
from keras import backend as K

def my_init(shape, dtype=None):
    return K.constant(shape=shape,value=(0,1,0),dtype=dtype)
    #return K.random_normal(shape, dtype=dtype)

def create_conv_model(X_train, Y_train, nb_filters = 1, nb_neighbors = 3, opt=None):
    
    nb_features = X_train.shape[1]
    data = Input(shape=(nb_features, 1), name="data", dtype=floatx())
    conv_layer = keras.layers.Conv1D(nb_neighbors, nb_filters, strides = nb_neighbors, 
                                     kernel_initializer=my_init)(data)#keras.initializers.Ones())(data)
    output = Flatten()(conv_layer)
    #output = Dense(units=Y_train.shape[1], activation="softmax", name='output')(flatt)

    model = Model(inputs=data, outputs=output)
    from keras import optimizers
    #opt = optimizers.Nadam(lr=1e-4)
    opt = optimizers.SGD(lr = 1e-4)
    model.compile(optimizer=opt, loss='categorical_crossentropy')

    return model

In [26]:
X = np.array([[1,1,1,1,1,1,1,1]], float)
Y = [1,0,1]
model = create_conv_model(X,Y)

In [27]:
X = X.reshape(1,8,1)

In [28]:
model.predict(X)

array([[ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.]], dtype=float32)